# 基础

1. 协议的概念
  
    
2. b/s c/s  

	- C/S  
        - 优点：   1. 协议选用灵活。  2. 缓存数据(从而提高传输效率)。

	     - 缺点：   1. 对用户的安全构成威胁  2. 开发工作量较大（client server），调试困难


	- B/S  
        - 优点：   1.工作量小（client）  2.不会对用户的安全构成威胁  3. 跨平台

	    -  缺点：   1. http（协议选择不灵活）   2.  不能提前缓存数据到客户机

	- 使用场景。

        - pad：    iphone 4   ipad 9    mac 11 13

	

3. 分层模型   7   4
    - 物 数 网 传 会 标 应
    - 网（网络接口或数据链路） 网（网络） 传 应
	

4. 协议格式

	- 数据包基本格式  数-应-传(TCP UDP)-网（IP）-链路（以太网帧，绑定mac地址，用来寻找下一跳路由器的作用）

	- 以太网帧格式
		`当前主机的mac地址 -- 下一跳的mac地址 -- 类型（【0800】【0806】arp请求 获取下一跳的mac） -- 数据（46-1500字节） -- crc
	arp数据包格式 --- arp攻击`

	- IP段格式 -- IP 32  
        - 网络层  封装的是IP地址（源、目的）
        - 版本号、首部长度、源、目的IP、生存时间（TTL）、数据

	- TCP/UDP  --  port 16  --- 65535（16位）   传输层
        - 源、目的端口号

5. NAT映射   打洞机制
    - 局域网IP映射到公网
    - 路由器保护 陌生ip第一次发送数据包会屏蔽， 公网ip 通过服务器为两个私有ip打洞

6. 套接字

	1. 成对出现  
        - 本质 借助Linux缓冲区形成的伪文件	
			
	2. bind  IP port

	3. 一个fd 两个buf  

7. TCP  

	- 预备知识。

		- "192.168.1.24" --》unsigned int -》 htonl() -》 网络字节序	【大端】

          -  10101010101001  -->　１０１０１０１０１０１００１０１
	
		- 192.168.1.24 ------------------> 网络字节序  inet_pton();

		- 网络字节序 ------------------> 点分十进制字符串  inet_ntop();

	- struct sockaddr_in addr;
		- 定义是使用，bind 等要转换成struct sockaddr *


```
	- addr.sin_family = AF_INET/AFINET6;地址簇
	- addr.sin_port = htons/ntohs;
	- addr.sin_addr.s_addr = htonl; ntosl（数值类型与网络字节序）;  inet_pton  inet_ntop（字符串的点分十进制与二进制）;
		
	- bind( , （struct sockaddr *）&addr);
      
      
	- accept();

	- connect();
 
 ```

- C/S模型
	- server.c

		1. socket()  建立套接字

		2. bind() 绑定IP 端口号  (struct sockaddr_in addr 初始化)

		3. listen() 指定最大同时发起连接数

		4. accept()  阻塞等待客户端发起连接  调用成功返回一个新的文件描述符，指向客户端的socket（connect传进来的那个socket）

		5. read()
		
		6. 小--大

		7. write 给 客户端

		8. close();

	- client.c

		1. socket();

		2. bind();  可以依赖“隐式绑定”

		3. connect();发起连接

		4. write();
		
		5. read();

		6. close();

# 笔记2

## 主要内容
1. client.c  编写
2. TCP三次握手建立连接
    - TCP——不会丢包（错！！！）  丢了可以重传。  
3. MTU、mss、半关闭
    - MTU： 最大传输单元    受协议限制   以太网1500   IP 65535
    - mss： 受MTU 标示一个数据包携带数据的上限数。 
    - win： 滑动窗口——当前本端 能接收的数据上限值。(单位：字节)
    - 半关闭：连接的双方只有一方发送了FIN关闭请求并收到了应答
4. 错误处理
    - read 返回值：
        -  `> 0 实际读到的字节数 buf=1024  1.  == buf 1024   2. < buf  56；`
        -  = 0 数据读完(读到文件、管道、socket 末尾--对端关闭) 
        -  -1  异常
        -  errno == EINTR  被信号中断  重启/quit
        -  errno == EAGAIN (EWOULDBLOCK) 非阻塞方式读，并且没有数据
        -  其他值   出现错误。--perror exit。
	- readn
	- readline
	- 错误一般思想  Accept  首字母大写，新函数，封装错误处理
5. 并发服务器模型
	- 多进程并发
        - fork   signal --- sigcation
	- 多线程并发
		- 线程分离 detach 。 修改线程属性	


## client.c
``` cpp
#include <stdio.h>  
#include <unistd.h>  
#include <stdlib.h>  
#include <sys/socket.h>  
#include <arpa/inet.h>  
#include <string.h>  
  
#define SERV_IP "192.168.15.129"  
#define SERV_PORT 6666  
	  
	int main(void){  
	    int cfd;  
	    struct sockaddr_in serv_addr;  
	    char buf[BUFSIZ];  
	    int n;  
	    socklen_t serv_addr_len;  
	          
	     /*创建一个socket 指定IPv4 TCP*/  
	    cfd = socket(AF_INET, SOCK_STREAM, 0);  
	      
	    /*初始化一个地址结构:*/  
	    memset(&serv_addr, 0, sizeof(serv_addr));           //清零  
	    serv_addr.sin_family = AF_INET;                    //IPv4协议族  
	    serv_addr.sin_port = htons(SERV_PORT);            //指定IP 字符串类型转换为网络字节序 参3:传出参数  
	    inet_pton(AF_INET, SERV_IP, &serv_addr.sin_addr.s_addr); //指定端口 本地转网络字节序  
	      
	    /*根据地址结构链接指定服务器进程*/   
	    connect(cfd, (struct sockaddr *)&serv_addr, sizeof(serv_addr));  
	      
	    while(1){  
	        /*从标准输入获取数据*/  
	        fgets(buf, sizeof(buf), stdin); //hello world == hello world\n\0  
	        /*将数据写给服务器*/  
	        write(cfd, buf, strlen(buf));  
	        n = read(cfd, buf, sizeof(buf));  
	        /*写至标准输出*/  
	        write(STDOUT_FILENO, buf, n);  
	    }  
	    /*关闭链接*/  
	    close(cfd);  
	    return 0;     
	}  

```

## SERVER.C
``` cpp
#include <stdio.h>  
#include <unistd.h>  
#include <sys/socket.h>  
#include <stdlib.h>  
#include <arpa/inet.h>  
#include <ctype.h>  
  
#define SERV_IP "127.0.0.1"  
#define SERV_PORT 6666  
	  
int main(void)  
{  
        int sfd, cfd;  
        int len, i;  
        char buf[BUFSIZ],clie_IP[BUFSIZ];  

        struct sockaddr_in serv_addr, clie_addr;  
        socklen_t clie_addr_len;  

        /*创建一个socket 指定IPv4协议族 TCP协议*/  
        sfd = socket(AF_INET, SOCK_STREAM, 0);  

        /*初始化一个地址结构 man 7 ip 查看对应信息*/  
        bzero(&serv_addr, sizeof(serv_addr));//将整个结构体清零  
        serv_addr.sin_family = AF_INET;     //选择协议族为IPv4  
        serv_addr.sin_addr.s_addr = htonl(INADDR_ANY);//监听本地所有IP地址  
        serv_addr.sin_port = htons(SERV_PORT);//绑定端口号     

        /*绑定服务器地址结构*/  
        bind(sfd, (struct sockaddr *)&serv_addr, sizeof(serv_addr));  

        /*设定链接上限,注意此处不阻塞*/  
        listen(sfd, 128);//同一时刻允许向服务器发起链接请求的数量  

        printf("wait for client connect ...\n");  
        /*获取客户端地址结构大小*/   
        clie_addr_len = sizeof(clie_addr_len);  
        /*参数1是sfd; 参2传出参数, 参3传入传入参数, 全部是client端的参数*/  
        cfd = accept(sfd, (struct sockaddr *)&clie_addr, &clie_addr_len);  

        printf("client IP:%s\tport:%d\n",  
                inet_ntop(AF_INET, &clie_addr.sin_addr.s_addr, clie_IP, sizeof(clie_IP)),  
                ntohs(clie_addr.sin_port));  
        while(1){  
             /*读取客户端发送数据*/  
                len = read(cfd,buf,sizeof(buf));  
                /*处理客户端数据*/  
                for(i=0; i<len; i++){  
                        buf[i]= toupper(buf[i]);  
                }  
                /*处理完数据回写给客户端*/  
                write(cfd, buf, len);  
        }  

        /*关闭链接*/  
        close(sfd);  
        close(cfd);  

        return 0;  
}  
```


![image1.png](https://i.loli.net/2018/08/09/5b6ba2c484019.png)

## 错误处理
### server.cpp
``` cpp
#include <stdio.h>  
#include <unistd.h>  
#include <sys/socket.h>  
#include <stdlib.h>  
#include <arpa/inet.h>  
#include <ctype.h>  
#include <string.h>  
  
#define SERV_IP "127.0.0.1"  
	#define SERV_PORT 6666  
	  
	int main(void)  
	{  
	        int sfd, cfd;  
	        int len, i;  
	        char buf[BUFSIZ],clie_IP[BUFSIZ];  
	  
	        struct sockaddr_in serv_addr, clie_addr;  
	        socklen_t clie_addr_len;  
	  
	        /*创建一个socket 指定IPv4协议族 TCP协议*/  
	        sfd = socket(AF_INET, SOCK_STREAM, 0);  
	        if(sfd == -1){  
	            perror("socket error");  
	            exit(1);  
	        }  
	  
	        /*初始化一个地址结构 man 7 ip 查看对应信息*/  
	        bzero(&serv_addr, sizeof(serv_addr));//将整个结构体清零  
	        serv_addr.sin_family = AF_INET;     //选择协议族为IPv4  
	        serv_addr.sin_addr.s_addr = htonl(INADDR_ANY);//监听本地所有IP地址  
	        serv_addr.sin_port = htons(SERV_PORT);//绑定端口号     
	  
	        /*绑定服务器地址结构*/  
	        int ret = bind(sfd, (struct sockaddr *)&serv_addr, sizeof(serv_addr));  
	        if(ret == -1){  
	            perror("bind error");  
	            exit(1);  
	        }  
	  
	        /*设定链接上限,注意此处不阻塞*/  
	        ret = listen(sfd, 128);//同一时刻允许向服务器发起链接请求的数量  
	        if(ret == -1){  
	            perror("listen error");  
	            exit(1);  
	        }  
	  
	        printf("wait for client connect ...\n");  
	        /*获取客户端地址结构大小*/   
	        clie_addr_len = sizeof(clie_addr_len);  
	        /*参数1是sfd; 参2传出参数, 参3传入传入参数, 全部是client端的参数*/  
	        cfd = accept(sfd, (struct sockaddr *)&clie_addr, &clie_addr_len);  
	          
	        if(cfd == -1){  
	            perror("accept error");  
	            exit(1);  
	        }  
	  
	        printf("client IP:%s\tport:%d\n",  
	                inet_ntop(AF_INET, &clie_addr.sin_addr.s_addr, clie_IP, sizeof(clie_IP)),  
	                ntohs(clie_addr.sin_port));  
	        while(1){  
	             /*读取客户端发送数据*/  
	                len = read(cfd,buf,sizeof(buf));  
	                  
	                /*处理客户端数据*/  
	                for(i=0; i<len; i++){  
	                        buf[i]= toupper(buf[i]);  
	                }  
	                /*处理完数据回写给客户端*/  
	                write(cfd, buf, len);  
	        }  
	          
	        /*关闭链接*/  
	        close(sfd);  
	        close(cfd);  
	  
	        return 0;  
	}  

```


- 第23 36 54行添加了判断，给出错误提示，但是此种错误处理会使程序混乱

### 错误处理2

- 另一种方法是
    - 对原先的socket bind accept等函数进行封装，在封装中加入错误判断
### wrap.c

``` cpp 
#include <stdlib.h>  
#include <stdio.h>  
#include <unistd.h>  
#include <errno.h>  
#include <sys/socket.h>  
  
void perr_exit(const char *s)  
{  
    perror(s);  
    exit(-1);  
}  
  
int Accept(int fd, struct sockaddr *sa, socklen_t *salenptr)  
{  
    int n;  
  
again:  
    if ((n = accept(fd, sa, salenptr)) < 0) {  
        if ((errno == ECONNABORTED) || (errno == EINTR))  
            goto again;  
        else  
            perr_exit("accept error");  
    }  
    return n;  
}  
  
int Bind(int fd, const struct sockaddr *sa, socklen_t salen)  
{  
    int n;  
  
    if ((n = bind(fd, sa, salen)) < 0)  
        perr_exit("bind error");  
  
    return n;  
}  
  
int Connect(int fd, const struct sockaddr *sa, socklen_t salen)  
{  
    int n;  
    n = connect(fd, sa, salen);  
    if (n < 0) {  
        perr_exit("connect error");  
    }  
  
    return n;  
}  
  
int Listen(int fd, int backlog)  
{  
    int n;  
  
    if ((n = listen(fd, backlog)) < 0)  
        perr_exit("listen error");  
  
    return n;  
}  
  
int Socket(int family, int type, int protocol)  
{  
    int n;  
  
    if ((n = socket(family, type, protocol)) < 0)  
        perr_exit("socket error");  
  
    return n;  
}  
  
ssize_t Read(int fd, void *ptr, size_t nbytes)  
{  
    ssize_t n;  
  
again:  
    if ( (n = read(fd, ptr, nbytes)) == -1) {  
        if (errno == EINTR)  
            goto again;  
        else  
            return -1;  
    }  
  
    return n;  
}  
  
ssize_t Write(int fd, const void *ptr, size_t nbytes)  
{  
    ssize_t n;  
  
again:  
    if ((n = write(fd, ptr, nbytes)) == -1) {  
        if (errno == EINTR)  
            goto again;  
        else  
            return -1;  
    }  
    return n;  
}  
  
int Close(int fd)  
{  
    int n;  
	    if ((n = close(fd)) == -1)  
	        perr_exit("close error");  
	  
	    return n;  
	}  
	  
	/*参三: 应该读取的字节数*/                          //socket 4096  readn(cfd, buf, 4096)   nleft = 4096-1500  
	ssize_t Readn(int fd, void *vptr, size_t n)  
	{  
	    size_t  nleft;              //usigned int 剩余未读取的字节数  
	    ssize_t nread;              //int 实际读到的字节数  
	    char   *ptr;  
	  
	    ptr = vptr;  
	    nleft = n;                  //n 未读取字节数  
	  
	    while (nleft > 0) {  
	        if ((nread = read(fd, ptr, nleft)) < 0) {  
	            if (errno == EINTR)  
	                nread = 0;  
	            else  
	                return -1;  
	        } else if (nread == 0)  
	            break;  
	  
	        nleft -= nread;   //nleft = nleft - nread   
	        ptr += nread;  
	    }  
	    return n - nleft;  
	}  
	  
	ssize_t Writen(int fd, const void *vptr, size_t n)  
	{  
	    size_t nleft;  
	    ssize_t nwritten;  
	    const char *ptr;  
	  
	    ptr = vptr;  
	    nleft = n;  
	    while (nleft > 0) {  
	        if ( (nwritten = write(fd, ptr, nleft)) <= 0) {  
	            if (nwritten < 0 && errno == EINTR)  
	                nwritten = 0;  
	            else  
	                return -1;  
	        }  
	        nleft -= nwritten;  
	        ptr += nwritten;  
	    }  
	    return n;  
	}  
	  
	static ssize_t my_read(int fd, char *ptr)  
	{  
	    static int read_cnt;  
	    static char *read_ptr;  
	    static char read_buf[100];  
	  
	    if (read_cnt <= 0) {  
	again:  
	        if ( (read_cnt = read(fd, read_buf, sizeof(read_buf))) < 0) {   //"hello\n"  
	            if (errno == EINTR)  
	                goto again;  
	            return -1;  
	        } else if (read_cnt == 0)  
	            return 0;  
	  
	        read_ptr = read_buf;  
	    }  
	    read_cnt--;  
	    *ptr = *read_ptr++;  
	  
	    return 1;  
	}  
	  
	/*readline --- fgets*/      
	//传出参数 vptr  
	ssize_t Readline(int fd, void *vptr, size_t maxlen)  
	{  
	    ssize_t n, rc;  
	    char    c, *ptr;  
	    ptr = vptr;  
	  
	    for (n = 1; n < maxlen; n++) {  
	        if ((rc = my_read(fd, &c)) == 1) {   //ptr[] = hello\n  
	            *ptr++ = c;  
	            if (c == '\n')  
	                break;  
	        } else if (rc == 0) {  
	            *ptr = 0;  
	            return n-1;  
	        } else  
	            return -1;  
	    }  
	    *ptr = 0;  
	  
	    return n;  
	}  

```
### wrap.h

``` cpp
#ifndef __WRAP_H_  
#define __WRAP_H_  
  
void perr_exit(const char *s);  
int Accept(int fd, struct sockaddr *sa, socklen_t *salenptr);  
int Bind(int fd, const struct sockaddr *sa, socklen_t salen);  
int Connect(int fd, const struct sockaddr *sa, socklen_t salen);  
int Listen(int fd, int backlog);  
int Socket(int family, int type, int protocol);  
	ssize_t Read(int fd, void *ptr, size_t nbytes);  
	ssize_t Write(int fd, const void *ptr, size_t nbytes);  
	int Close(int fd);  
	ssize_t Readn(int fd, void *vptr, size_t n);  
	ssize_t Writen(int fd, const void *vptr, size_t n);  
	ssize_t my_read(int fd, char *ptr);  
	ssize_t Readline(int fd, void *vptr, size_t maxlen);  
	  
	#endif  
```

### server.c

``` cpp
#include <stdio.h>  
#include <unistd.h>  
#include <sys/types.h>  
#include <sys/socket.h>  
#include <strings.h>  
#include <string.h>  
#include <ctype.h>  
#include <arpa/inet.h>  
  
	#include "wrap.h"  
	  
	#define SERV_PORT 6666  
	  
	int main(void)  
	{  
	    int sfd, cfd;  
	    int len, i;  
	    char buf[BUFSIZ], clie_IP[BUFSIZ];  
	  
	    struct sockaddr_in serv_addr, clie_addr;  
	    socklen_t clie_addr_len;  
	  
	    sfd = Socket(AF_INET, SOCK_STREAM, 0);  
	  
	    bzero(&serv_addr, sizeof(serv_addr));             
	    serv_addr.sin_family = AF_INET;                   
	    serv_addr.sin_addr.s_addr = htonl(INADDR_ANY);    
	    serv_addr.sin_port = htons(SERV_PORT);            
	  
	    Bind(sfd, (struct sockaddr *)&serv_addr, sizeof(serv_addr));  
	  
	    Listen(sfd, 2);                                  
	  
	    printf("wait for client connect ...\n");  
	  
	    clie_addr_len = sizeof(clie_addr_len);  
	    cfd = Accept(sfd, (struct sockaddr *)&clie_addr, &clie_addr_len);  
	    printf("cfd = ----%d\n", cfd);  
	  
	    printf("client IP: %s  port:%d\n",   
	            inet_ntop(AF_INET, &clie_addr.sin_addr.s_addr, clie_IP, sizeof(clie_IP)),   
	            ntohs(clie_addr.sin_port));  
	  
	    while (1) {  
	        len = Read(cfd, buf, sizeof(buf));  
	        Write(STDOUT_FILENO, buf, len);  
	  
	        for (i = 0; i < len; i++)  
	            buf[i] = toupper(buf[i]);  
	        Write(cfd, buf, len);   
	    }  
	  
	    Close(sfd);  
	    Close(cfd);  
	  
	    return 0;  
	}  

```

### client.c

``` cpp
#include <stdio.h>  
#include <unistd.h>  
#include <string.h>  
#include <sys/socket.h>  
#include <arpa/inet.h>  
  
#include "wrap.h"  
  
#define SERV_IP "127.0.0.1"  
#define SERV_PORT 6666  
  
int main(void)  
{  
    int sfd, len;  
    struct sockaddr_in serv_addr;  
    char buf[BUFSIZ];   
  
    sfd = Socket(AF_INET, SOCK_STREAM, 0);  
  
    bzero(&serv_addr, sizeof(serv_addr));                         
    serv_addr.sin_family = AF_INET;                               
    inet_pton(AF_INET, SERV_IP, &serv_addr.sin_addr.s_addr);      
    serv_addr.sin_port = htons(SERV_PORT);                        
  
    Connect(sfd, (struct sockaddr *)&serv_addr, sizeof(serv_addr));  
  
    while (1) {  
        fgets(buf, sizeof(buf), stdin);  
        int r = Write(sfd, buf, strlen(buf));         
        printf("Write r ======== %d\n", r);  
        len = Read(sfd, buf, sizeof(buf));  
        printf("Read len ========= %d\n", len);  
        Write(STDOUT_FILENO, buf, len);  
    }  
  
    Close(sfd);  
  
    return 0;  
}  

```



### makefile
    - 命名 makefile 或Makefile
    - 规则    目标 、 依赖、 命令
    - 目标：依赖条件	//第一行
    - tab键 命令	//第二行
    - 版本1
    
![image2.png](https://i.loli.net/2018/08/09/5b6ba41b9e7d2.png)
- 版本2
    - 为了节约时间 ，若只修改一个文件，不需要对全部文件重新编译
    - 提高了效率
    - 生成终极目标的规则放在最上面
    
![image3.png](https://i.loli.net/2018/08/09/5b6ba44e29da0.png)
![image4.png](https://i.loli.net/2018/08/09/5b6ba44c64199.png)
![image5.png](https://i.loli.net/2018/08/09/5b6ba44ac1417.png)
- 版本3
    - 版本2冗余过多
![image6.png](https://i.loli.net/2018/08/09/5b6ba4a8d18fd.png)
![image7.png](https://i.loli.net/2018/08/09/5b6ba4a77e422.png)
![image8.png](https://i.loli.net/2018/08/09/5b6ba4a5dac69.png)

```
src = $(wildcard *.c)  
obj = $(patsubst %.c, %.o, $(src))  
  
all: server client  
  
server: server.o wrap.o  
    gcc server.o wrap.o -o server -Wall  
client: client.o wrap.o  
    gcc client.o wrap.o -o client -Wall  
	  
%.o:%.c  
    gcc -c $< -Wall  

.PHONY: clean all  
clean:   
    -rm -rf server client $(obj)  
```



### gdb调试

``` c
//gdb 测试  
//$ gcc *.c -o test -g === 第二个参数选择文件，第三个参数输出的文件，第四个参数调试  
//$ gdb test == 开始调试  
//$ l === 查看代码  没有参数表示查看main函数  
//$ l === 继续查看，直接回车表示重复上次代码  
//$ l 某个c文件:数字/函数名 === 查看某个c文件的第几行或函数  
//$ break 数字 === 在某一行添加断点  简写b也可以  
//$ b 数字 if i==15 === 条件断点 在某行产生中断，当i等于15时  
//$ i b === 查看断点信息  Enb 为y表示断点可用  
//$ start === 开始执行程序，只执行一步  
//$ n === 单步执行  
//$ c === 继续执行 到断点  
//$ s === 进入函数内部  
//$ p 变量 === 查看变量的值 如p arr[5]  
//$ ptype 变量 === 查看变量类型  
//$ display 变量 === 追踪变量的值  
//$ info display === 获取追踪变量的编号  
//$ undisplay 数字 === 这个数字为：取消追踪变量的编号  
//$ u === 跳出单次循环  
//$ finish === 跳出当前函数  可能需要先删除断点  
//$ info break === 查看断点编号  
//$ d 数字 === 删除断点 数字为：断点编号  
//$ set var i=10 === 循环是设置，观察i=10 的情况  
//$ quit === 退出gdb  

#include <iostream>  
#include <vector>  

using namespace std;  

bool cmp(int a, int b){  
    return a<b;  
}  

int main(void){  
    int a=1;  
    int b=2;  

    int ret =0;  

    ret = cmp(a,b);  

    cout<<"a<b?\t"<<(bool)(ret)<<endl;  

    return 0;  
}  

```


## 三次握手四次握手

![image9.png](https://i.loli.net/2018/08/09/5b6ba55da0add.png)

## 多进程

### 使用多进程并发服务器时要考虑以下几点：

- 父进程最大文件描述个数(父进程中需要close关闭accept返回的新文件描述符)
- 系统内创建进程个数(与内存大小相关)
- 进程创建过多是否降低整体服务性能(进程调度)
- 僵尸进程   子进程交互完后exit变为僵尸进程
- waitpid();回收  只能由父进程回收
- 父进程一直在揽活（与新客户端进行连接）
	- 接收连接的交给子进程做，父进程用来回收
	- 用线程
	- 守护进程
	- 回调机制  注册一个SIGCHLD的信号捕捉函数，在捕捉函数里面把waitpid调用上，内核处理
    
### 僵尸进程

- 僵尸进程就是指：一个进程执行了exit系统调用退出，而其父进程并没有为它收尸(调用wait或waitpid来获得它的结束状态)的进程。
- 如何避免僵尸进程？
	- 通过signal(SIGCHLD, SIG_IGN)通知内核对子进程的结束不关心，由内核回收。如果不想让父进程挂起，可以在父进程中加入一条语句：signal(SIGCHLD,SIG_IGN);表示父进程忽略SIGCHLD信号，该信号是子进程退出的时候向父进程发送的。
	- 父进程调用wait/waitpid等函数等待子进程结束，如果尚无子进程退出wait会导致父进程阻塞。waitpid可以通过传递WNOHANG使父进程不阻塞立即返回。
	- 如果父进程很忙可以用signal注册信号处理函数，在信号处理函数调用wait/waitpid等待子进程退出。
	- 通过两次调用fork。父进程首先调用fork创建一个子进程然后waitpid等待子进程退出，子进程再fork一个孙进程后退出。这样子进程退出后会被父进程等待回收，而对于孙子进程其父进程已经退出所以孙进程成为一个孤儿进程，孤儿进程由init进程接管，孙进程结束后，init会等待回收。

- 第一种方法忽略SIGCHLD信号，这常用于并发服务器的性能的一个技巧因为并发服务器常常fork很多子进程，子进程终结之后需要服务器进程去wait清理资源。如果将此信号的处理方式设为忽略，可让内核把僵尸子进程转交给init进程去处理，省去了大量僵尸进程占用系统资源。

#### 僵尸进程处理办法

- 1 wait()函数

```
1.	#include <sys/types.h> 
2.	#include <sys/wait.h>
3.	pid_t wait(int *status);
```
- 进程一旦调用了wait，就立即阻塞自己，由wait自动分析是否当前进程的某个子进程已经退出，如果让它找到了这样一个已经变成僵尸的子进程，wait就会收集这个子进程的信息，并把它彻底销毁后返回；如果没有找到这样一个子进程，wait就会一直阻塞在这里，直到有一个出现为止。
- 参数status用来保存被收集进程退出时的一些状态，它是一个指向int类型的指针。但如果我们对这个子进程是如何死掉的毫不在意，只想把这个僵尸进程消灭掉，（事实上绝大多数情况下，我们都会这样想），我们就可以设定这个参数为NULL，就象下面这样：
    `pid = wait(NULL);`
- 如果成功，wait会返回被收集的子进程的进程ID，如果调用进程没有子进程，调用就会失败，此时wait返回-1，同时errno被置为ECHILD。
	- wait系统调用会使父进程暂停执行，直到它的一个子进程结束为止。
	- 返回的是子进程的PID，它通常是结束的子进程
	- 状态信息允许父进程判定子进程的退出状态，即从子进程的main函数返回的值或子进程中exit语句的退出码。
	- 如果status不是一个空指针，状态信息将被写入它指向的位置
- 可以上述的一些宏判断子进程的退出情况：

![image10.png](https://i.loli.net/2018/08/09/5b6ba55df1306.png)

- 2 waitpid()函数 

```cpp
#include <sys/types.h> 
#include <sys/wait.h>
pid_t waitpid(pid_t pid, int *status, int options);
```
- 参数:
- status:如果不是空，会把状态信息写到它指向的位置，与wait一样
- options：允许改变waitpid的行为，最有用的一个选项是WNOHANG,它的作用是防止waitpid把调用者的执行挂起
- The value of options is an OR of zero or more  of  the  following  constants:
- WNOHANG     return immediately if no child has exited.
- WUNTRACED   also  return  if  a  child  has stopped (but not traced via ptrace(2)).  Status for traced children which have  stopped  is provided even if this option is not specified.
- WCONTINUED (since Linux 2.6.10) also return if a stopped child has been resumed by delivery of SIGCONT.
- 返回值：如果成功返回等待子进程的ID，失败返回-1
- 对于waitpid的pid参数的解释与其值有关：
    - pid == -1 等待任一子进程。于是在这一功能方面waitpid与wait等效。
    - pid > 0 等待其进程I D与p i d相等的子进程。
    - pid == 0 等待其组I D等于调用进程的组I D的任一子进程。换句话说是与调用者进程同在一个组的进程。
    - pid < -1 等待其组I D等于p i d的绝对值的任一子进程
- wait与waitpid区别：
	- 在一个子进程终止前， wait 使其调用者阻塞，而waitpid 有一选择项，可使调用者不阻塞。
	- waitpid并不等待第一个终止的子进程—它有若干个选择项，可以控制它所等待的特定进程。
	- 实际上wait函数是waitpid函数的一个特例。waitpid(-1, &status, 0);
- 示例：
- 如以下代码会创建100个子进程，但是父进程并未等待它们结束，所以在父进程退出前会有100个僵尸进程。

``` cpp
#include <stdio.h>  
#include <unistd.h>  
   
int main() {  
   
  int i;  
  pid_t pid;  
   
  for(i=0; i<100; i++) {  
	    pid = fork();  
	    if(pid == 0)  
	      break;  
	  }  
	   
	  if(pid>0) {  
	    printf("press Enter to exit...");  
	    getchar();  
	  }  
	   
	  return 0;  
	}
```

- 其中一个解决方法即是编写一个SIGCHLD信号处理程序来调用wait/waitpid来等待子进程返回。

``` cpp
#include <stdio.h>  
#include <unistd.h>  
#include <signal.h>  
#include <sys/types.h>  
#include <sys/wait.h>  
   
void wait4children(int signo) {  
   
  int status;  
	  wait(&status);  
	   
	}  
	   
	int main() {  
	  int i;  
	  pid_t pid;  
	   
	  signal(SIGCHLD, wait4children);  
	   
	  for(i=0; i<100; i++) {  
	    pid = fork();  
	    if(pid == 0)  
	      break;  
	  }  
	   
	  if(pid>0) {  
	    printf("press Enter to exit...");  
	    getchar();  
	  }  
	  return 0;  
	}  
```

- 但是通过运行程序发现还是会有僵尸进程，而且每次僵尸进程的数量都不定。这是为什么呢？其实主要是因为Linux的信号机制是不排队的，假如在某一时间段多个子进程退出后都会发出SIGCHLD信号，但父进程来不及一个一个地响应，所以最后父进程实际上只执行了一次信号处理函数。但执行一次信号处理函数只等待一个子进程退出，所以最后会有一些子进程依然是僵尸进程。
- 虽然这样但是有一点是明了的，就是收到SIGCHLD必然有子进程退出，而我们可以在信号处理函数里循环调用waitpid函数来等待所有的退出的子进程。至于为什么不用wait，主要原因是在wait在清理完所有僵尸进程后再次等待会阻塞。

- 所以最佳方案如下：

``` cpp
#include <stdio.h>  
#include <unistd.h>  
#include <signal.h>  
#include <errno.h>  
#include <sys/types.h>  
#include <sys/wait.h>  
   
void wait4children(int signo) {  
  int status;  
  while(waitpid(-1, &status, WNOHANG) > 0);  
}  
   
int main() {  
   
  int i;  
  pid_t pid;  
   
  signal(SIGCHLD, wait4children);  
   
  for(i=0; i<100; i++) {  
    pid = fork();  
    if(pid == 0)  
      break;  
  }  
   
  if(pid>0) {  
    printf("press Enter to exit...");  
    getchar();  
  }  
   
  return 0;  
}  
```

- 这里使用waitpid而不是使用wait的原因在于：我们在一个循环内调用waitpid，以获取所有已终止子进程的状态。我们必须指定WNOHANG选项，它告诉waitpid在有尚未终止的子进程在运行时不要阻塞。我们不能在循环内调用wait，因为没有办法防止wait在正运行的子进程尚有未终止时阻塞。
- 父进程与子进程区分
- 在子进程中，fork函数返回0，在父进程中，fork返回新创建子进程的进程ID。

### server.c

```cpp
#include <stdio.h>  
#include <unistd.h>  
#include <stdlib.h>  
#include <arpa/inet.h>  
#include <ctype.h>  
#include <strings.h>  
#include <netinet/in.h>  
#include <signal.h>  
#include <sys/wait.h>  
  
#include "wrap.h"  
  
#define MAXLINE 8192  
#define SERV_PORT 8888  
  
void wait_child(int signo)  
{  
    while(waitpid(0, NULL, WNOHANG));  
    return ;  
}  
  
int main(void)  
{  
    pid_t pid;  
    int lfd, cfd;  
    struct sockaddr_in serv_addr, clie_addr;  
    socklen_t clie_addr_len;  
    char buf[BUFSIZ];  
    int i;  
      
    lfd = Socket(AF_INET, SOCK_STREAM, 0);  
    bzero(&serv_addr, sizeof(serv_addr));  
    serv_addr.sin_family = AF_INET;  
    serv_addr.sin_port = htons(SERV_PORT);  
    serv_addr.sin_addr.s_addr = htonl(INADDR_ANY);  
    //inet_pton(AF_INET, "192.168.1.1", &serv_addr.sin_addr_s_addr);  
    Bind(lfd, (struct sockaddr *)&serv_addr, sizeof(serv_addr));  
      
    Listen(lfd, 128);  
    while(1){  
        clie_addr_len = sizeof(clie_addr);  
        cfd = Accept(lfd, (struct sockaddr *)&clie_addr, &clie_addr_len);  
          
        pid = fork();  
        if(pid < 0){  
            perror("fork error");  
            exit(1);  
        }else if(pid == 0){  
            close(lfd);  
            break;  
        }else { //父进程  
            close(cfd);  
            signal(SIGCHLD, wait_child);  
        }  
    }  
      
      
    if(pid == 0){  
        while(1){  
            int n = read(cfd, buf, sizeof(buf));  
            if(n == 0){     //说明client 关闭了  
                close(cfd);  
                return 0;  
            }else if(n == -1){  //可能被信号打断  
                perror("read error");  
                exit(1);  
            }else{  
                for(i=0; i<n; i++){  
                    buf[i] = toupper(buf[i]); //ctype.h`  
                }  
                write(cfd, buf, n);  
            }  
        }  
    }  
      
}  
```

## 多线程

### 多线程基本概念

- 线程资源
- 由于一个程序中的多个线程共享同一地址空间，因此代码段，数据段内容是共享的。除此之外，以下内容也是共享的：
    - 文件描述符表
    - 每种信号的处理方式(SIG_IGN、SIG_DFL或者自定义的信号处理函数)
    - 当前工作目录
    - 用户id和组id
- 但有些资源是每个线程各有一份的:
	-  线程id
	-  上下文,包括各种寄存器的值、程序计数器和栈指针
	-  栈空间
	-  errno变量
	-  信号屏蔽字
	-  调度优先级
- 线程控制
    - 创建：
    `int pthread_create(pthread_t *thread, const pthread_attr_t *attr,void *(*start_routine) (void *), void *arg);`
- 获取线程id：
    `pthread_t pthread_self(void);`
- 终止线程
- 如果需要只终止某个线程而不终止整个进程,可以有三种方法:
	-  从线程函数return。这种方法对主线程不适用,从main函数return相当于调用exit。//线程return
	-  一个线程可以调用pthread_cancel终止同一进程中的另一个线程。//被动终止
	-  线程可以调用pthread_exit终止自己。//自己终止

![image11.png](https://i.loli.net/2018/08/09/5b6ba5609a83d.png)

- 等待线程
    ` int pthread_join(pthread_t thread, void **retval);`
- 调用该函数等待的线程将挂起等待，直到id为thread的线程终止。线程以不同方式终止，pthread_join得到的结果是不同的。具体如下：
    - 线程以return方式终止，那么retval指向的单元保存的是thread函数的返回值
    - 如果进程被pthread_cancel方式终止，那么retval保存的是常数PTHREAD_CANCELED
    - 如果进程是pthread_exit方式终止自己，那么reval保存用户传给exit的参数
- 注：对于线程进行join之后线程的状态将是detach状态（分离），同样的pthread_cancel函数可以对线程进行分离处理。所以，不能同时对一个线程进行join和detach操作
- 线程分离
- 在任意一个时间点上，线程是可结合（joinable）或者是可分离的（detached）。一个可结合线程是可以被其他线程收回资源和杀死的。在被回收之前，他的存储器资源（栈等）是不释放的。而对于detached状态的线程，其资源不能被别的线程收回和杀死，只有等到线程结束才能由系统自动释放
- 默认情况，线程状态被设置为结合的。所以为了避免资源泄漏等问题，一个线程应当是被显示的join或者detach的，否则线程的状态类似于进程中的Zombie Process。会有部分资源没有被回收的。
- 调用函数pthread_join，当等待线程没有终止时，主线程将处于阻塞状态。如果要避免阻塞，那么
    - 在主线程中加入代码pthread_detach(thread_id)
    - 或者在被等待线程中加入pthread_detach(thread_self())
![image12.png](https://i.loli.net/2018/08/09/5b6ba5604ad83.png)


### server.c

``` cpp
#include <stdio.h>  
#include <string.h>  
#include <arpa/inet.h>  
#include <pthread.h>  
#include <ctype.h>  
#include <unistd.h>  
#include <fcntl.h>  
  
#include "wrap.h"  
  
#define MAXLINE 8192  
#define SERV_PORT 8000  
  
struct s_info {                     //定义一个结构体, 将地址结构跟cfd捆绑  
    struct sockaddr_in cliaddr;  
    int connfd;  
};  
  
void *do_work(void *arg)  
{  
    int n,i;  
    struct s_info *ts = (struct s_info*)arg;  
    char buf[MAXLINE];  
    char str[INET_ADDRSTRLEN];      //#define INET_ADDRSTRLEN 16  可用"[+d"查看  
  
    while (1) {  
        n = Read(ts->connfd, buf, MAXLINE);                     //读客户端  
        if (n == 0) {  
            printf("the client %d closed...\n", ts->connfd);  
            break;                                              //跳出循环,关闭cfd  
        }  
        printf("received from %s at PORT %d\n",  
                inet_ntop(AF_INET, &(*ts).cliaddr.sin_addr, str, sizeof(str)),  
                ntohs((*ts).cliaddr.sin_port));                 //打印客户端信息(IP/PORT)  
  
        for (i = 0; i < n; i++)   
            buf[i] = toupper(buf[i]);                           //小写-->大写  
  
        Write(STDOUT_FILENO, buf, n);                           //写出至屏幕  
        Write(ts->connfd, buf, n);                              //回写给客户端  
    }  
    Close(ts->connfd);  
  
    return (void *)0;  
}  
  
int main(void)  
{  
    struct sockaddr_in servaddr, cliaddr;  
    socklen_t cliaddr_len;  
    int listenfd, connfd;  
    pthread_t tid;  
    struct s_info ts[256];      //根据最大线程数创建结构体数组.  
    int i = 0;  
  
    listenfd = Socket(AF_INET, SOCK_STREAM, 0);                     //创建一个socket, 得到lfd  
  
    bzero(&servaddr, sizeof(servaddr));                             //地址结构清零  
    servaddr.sin_family = AF_INET;  
    servaddr.sin_addr.s_addr = htonl(INADDR_ANY);                   //指定本地任意IP  
    servaddr.sin_port = htons(SERV_PORT);                           //指定端口号 8000  
  
    Bind(listenfd, (struct sockaddr *)&servaddr, sizeof(servaddr)); //绑定  
  
    Listen(listenfd, 128);      //设置同一时刻链接服务器上限数  
  
    printf("Accepting client connect ...\n");  
  
    while (1) {  
        cliaddr_len = sizeof(cliaddr);  
        connfd = Accept(listenfd, (struct sockaddr *)&cliaddr, &cliaddr_len);   //阻塞监听客户端链接请求  
        ts[i].cliaddr = cliaddr;  
        ts[i].connfd = connfd;  
  
        /* 达到线程最大数时，pthread_create出错处理, 增加服务器稳定性 */  
        pthread_create(&tid, NULL, do_work, (void*)&ts[i]);  
        pthread_detach(tid);                                                    //子线程分离,防止僵线程产生.  
        i++;  
    }  
  
    return 0;  
}  
```

